In [1]:
!pip install wget
import os
import torch
import pandas as pd
import numpy as np
import torch
from torch import nn


ssh_config = """
Host github.com
  IdentityFile ~/.ssh/github.pem
  User davipeag
  StrictHostKeyChecking no
"""

if os.name == 'nt':
  base_path = ""
  REPO_DIR = "."
  STORE_DIR ="." 
  print("Windows")
else:
  print("Unix-like")
  REPO_DIR = "/tmp/HeartRateRegression"
  from google.colab import drive
  drive.mount('/content/drive')
  GIT_PATH = "/content/drive/My\ Drive/deeplearning_project/github.pem"
  DATA_DIR = os.path.join(REPO_DIR, "repo")
  STORE_DIR ="/content/drive/My Drive/deeplearning_project/" 
  !mkdir ~/.ssh
  !cp -u {GIT_PATH} ~/.ssh/
  !chmod u=rw,g=,o= ~/.ssh/github.pem
  !echo "{ssh_config}" > ~/.ssh/config
  !chmod u=rw,g=,o= ~/.ssh/config
  ! (cd /tmp && git clone git@github.com:davipeag/HeartRateRegression.git)
  ! (cd {REPO_DIR} && git pull )
  import sys
  sys.path.append(REPO_DIR)

def git_pull():
  ! (cd {REPO_DIR} && git pull )

git_pull()

Unix-like
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.ssh’: File exists
fatal: destination path 'HeartRateRegression' already exists and is not an empty directory.
Already up to date.
Already up to date.


In [2]:
import random
import torch
from torch import nn
import numpy as np
args = {
    'epoch_num': 250,     # Number of epochs.
    'lr': 1.0e-3,           # Learning rate.
    'weight_decay': 10e-4, # L2 penalty.
    'momentum': 0.9,      # Momentum.
    'num_workers': 0,     # Number of workers on data loader.
    'batch_size': 128,     # Mini-batch size. 128
    'batch_test': 248,     # size of test batch
    'window': 15,
    'initial_window':5,
    'clip_norm': 6.0,     # Upper limit on gradient L2 norm ###
}
if torch.cuda.is_available():
    args['device'] = torch.device('cuda')
else:
    args['device'] = torch.device('cpu')

print(args['device'])

SEED = 1234
def reset_seeds():
  random.seed(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

reset_seeds()

cuda


In [3]:
from data_utils import (FormatIeee,  IeeeExtractor)

SUBJECTS = list(range(1,13))

extractor = IeeeExtractor(DATA_DIR)
formatter = FormatIeee()
dfs_train = [formatter.transform(extractor.extract_subject(i)) for i in SUBJECTS]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  import sys


In [4]:
git_pull()

import importlib

import PPG

from PPG import FullTrainer

importlib.reload(PPG.AttentionDefaults)
importlib.reload(PPG)
importlib.reload(PPG.UtilitiesDataXY)
importlib.reload(PPG.Models)
importlib.reload(PPG.NoHrPceLstmModel)
importlib.reload(PPG.TrainerXY)
importlib.reload(PPG.TrainerIS)
importlib.reload(PPG.FullTrainer)


Already up to date.


<module 'PPG.FullTrainer' from '/tmp/HeartRateRegression/PPG/FullTrainer.py'>

In [ ]:
import matplotlib.pyplot as plt
import pickle
# fchoice = {'val_sub': 4,
#   'ts_sub': 0,
#   'batch_size': 64,
#   'weight_decay': 0,
#   'lr': 0.001,
#   'nattrs': 5,
#   'bvp_count': 16,
#   'dropout_rate': 0.25,
#   'lstm_input': 128,
#   'lstm_size': 64,
#   'ts_h_size': 64
#   }
def compute_ensemble(results):
  ps = [v["predictions"][1].reshape(-1).numpy() for v in results]
  ys = [v["predictions"][0].reshape(-1).numpy() for v in results]

  for i in range(1, len(ys)-1):
    assert np.all(ys[i] == ys[i-1])

  s = ps[0]
  for p in ps[1:]:
    s = s + p

  a = s/len(ps)
  y = ys[0]

  plt.plot(a)
  plt.plot(y)

  return np.mean(np.abs(a - y))


fchoice = {'val_sub': 4,
  'ts_sub': 0,
  'batch_size': 64,
  'weight_decay': 0.0001,
  'lr': 0.001,
  'nattrs': 6,
  'bvp_count': 12,
  'dropout_rate': 0,
  'lstm_input': 128,
  'lstm_size': 32,
  'ts_h_size': 32
  }



from PPG import UtilitiesDataXY


aresults = list()
for ts_sub in [0,1,2,3, 4,5,6,7,8,9,10,11,12,13,14]:
  dresults = list()
  for i in range(7):
    filename = f"ieee_train_ts_{ts_sub}_{i}.pkl"
    save_path = os.path.join(STORE_DIR, filename)
    try:
      with open(save_path , "rb") as f:
        out = pickle.load(f)
    except FileNotFoundError:
      full_trainer = FullTrainer.IeeeJointValNoHrPceLstmFullTrainer(dfs_train, args["device"])
    else:
      dresults.append(out)
      continue
    # try:
    fchoice["ts_sub"] = ts_sub
    out = full_trainer.train(**fchoice)
    print(out["args"], out["metric"])
    dresults.append(out)
    with open(save_path, "wb") as f:
      results = pickle.dump(out, f)
    # except RuntimeError as e:
    #   if isinstance(e, KeyboardInterrupt):
    #     raise e
    #   else:
    #     print("####")
    #     print(f"Failed: {choice}")
    #     print("###")
  # print(f"TS:{compute_ensemble(dresults)}")
  # aresults.append(dresults)


best val epoch: 1
[1/40]: loss_train: 4.396 loss_val 5.246 loss_ts 10.122
best val epoch: 2
[2/40]: loss_train: 2.343 loss_val 3.737 loss_ts 7.522
best val epoch: 3
[3/40]: loss_train: 2.006 loss_val 3.473 loss_ts 6.645
best val epoch: 4
[4/40]: loss_train: 1.975 loss_val 3.430 loss_ts 5.582
best val epoch: 28
[28/40]: loss_train: 1.645 loss_val 3.368 loss_ts 2.790
best val epoch: 36
[36/40]: loss_train: 1.662 loss_val 3.327 loss_ts 2.883
best val epoch: 37
[37/40]: loss_train: 1.583 loss_val 3.213 loss_ts 2.849
best val epoch: 38
[38/40]: loss_train: 1.554 loss_val 3.165 loss_ts 2.824
Final: 2.8236024379730225
{'val_sub': 4, 'ts_sub': 0, 'batch_size': 64, 'weight_decay': 0.0001, 'lr': 0.001, 'nattrs': 6, 'bvp_count': 12, 'dropout_rate': 0, 'lstm_input': 128, 'lstm_size': 32, 'ts_h_size': 32} 2.8236024
best val epoch: 1
[1/40]: loss_train: 3.434 loss_val 3.124 loss_ts 4.148
best val epoch: 2
[2/40]: loss_train: 2.681 loss_val 2.594 loss_ts 2.951
best val epoch: 8
[8/40]: loss_train: 2.